In [1]:
!pip install -q wikipedia langchain openai faiss-cpu tiktoken

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 34.7 MB/s eta 0:00:00


In [2]:
!pip install -q transformers sentence-transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.0 MB/s eta 0:00:00


In [3]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.5 MB/s eta 0:00:00


In [4]:
import os

In [5]:
import wikipedia

# List of article titles (adjust to your needs)
titles = [
    "Water_supply_and_sanitation_in_India", "Water_supply", "Water_distribution_system", "Non-revenue_water", "Improved_water_source",
    "Water_scarcity", "Rainwater_harvesting", "Open_defecation",
    "Improved_sanitation", "Pit_latrine", "Sewage", "Wastewater_treatment",
    "Manual_scavenging", "Wastewater", "Sewage_treatment",
    "Community-led_total_sanitation", "Microcredit_for_water_supply_and_sanitation", "Swachh_Bharat_Mission",
    "Fecal_sludge_management", "Water_tariff"
]

# Download the content
documents = []
for title in titles:
    try:
        content = wikipedia.page(title).content
        documents.append((title, content))
    except Exception as e:
        print(f"Error loading {title}: {e}")

Error loading Non-revenue_water: Page id "non revenue water" does not match any pages. Try another id!
Error loading Water_scarcity: Page id "water scarity" does not match any pages. Try another id!
Error loading Open_defecation: Page id "open defection" does not match any pages. Try another id!


In [ ]:
documents

[('Water_supply_and_sanitation_in_India',
  'In 2018, 98.7% of Indians had access to the basic water and sanitation facilities. India faces challenges ranging from sourcing water for its megacities to its distribution network which is intermittent in rural areas with continuous distribution networks just beginning to emerge. Non-revenue water is a challenge.\nThe share of Indians with access to improved sources of water increased significantly from 72% in 1990 to 88% in 2008 and currently stands at 98.7% in 2018.  In 1980, rural sanitation coverage was estimated at 1%. By 2018, it reached over 98%.:\u200a78\u200a However, many people still lack access to water and sewage infrastructure.\n\n\n== Water supply ==\n\n\n=== Cities ===\nIn 2005, none of the 35 Indian cities with a population of more than one million distributed water for more than a few hours per day, despite generally sufficient infrastructure. Owing to inadequate water pressure people struggle to collect water even when it

In [6]:
with open("all_wikipedia_articles.txt", "w", encoding="utf-8") as f:
    for title, content in documents:
        f.write(f"=== {title} ===\n")
        f.write(content)
        f.write("\n\n")

print("All articles saved in 'all_wikipedia_articles.txt'")

All articles saved in 'all_wikipedia_articles.txt'


In [7]:
def clean_wiki_text(text):
    # List of unwanted section titles (case-insensitive match)
    unwanted_sections = [
        "See also", "References", "Further reading", "External links", "Bibliography", "Notes", "Sources"
    ]

    for section in unwanted_sections:
        split_marker = f"\n{section}"
        lower_split_marker = f"\n{section.lower()}"

        # Try matching both cases (some may be lowercased)
        if split_marker in text:
            text = text.split(split_marker)[0]
        elif lower_split_marker in text:
            text = text.split(lower_split_marker)[0]

    return text.strip()

In [8]:
# Cleaned documents
cleaned_documents = [(title, clean_wiki_text(content)) for title, content in documents]

In [9]:
with open("all_wikipedia_articles_cleaned.txt", "w", encoding="utf-8") as f:
    for title, content in cleaned_documents:
        f.write(f"=== {title} ===\n")
        f.write(content)
        f.write("\n\n")

print("Cleaned articles saved to 'all_wikipedia_articles_cleaned.txt'")

Cleaned articles saved to 'all_wikipedia_articles_cleaned.txt'


In [10]:
from typing import List
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

def chunk_text(title: str, text: str, max_tokens=300) -> List[dict]:
    paragraphs = text.split('\n')
    chunks = []
    for para in paragraphs:
        para = para.strip()
        if len(para.split()) > 30:
            chunks.append({"title": title, "text": para})
    return chunks

all_chunks = []
for title, content in cleaned_documents:
    all_chunks.extend(chunk_text(title, content))

print(f"Total chunks: {len(all_chunks)}")

Total chunks: 554


In [11]:
from sentence_transformers import SentenceTransformer
import numpy as np

model_name = 'sentence-transformers/all-MiniLM-L6-v2'  # Fast & lightweight
embedder = SentenceTransformer(model_name)

texts = [chunk["text"] for chunk in all_chunks]
embeddings = embedder.encode(texts, convert_to_numpy=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
import faiss

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)
print("FAISS index ready.")

FAISS index ready.


In [13]:
def search_similar_chunks(query, top_k=4):
    query_vector = embedder.encode([query])
    scores, indices = index.search(query_vector, top_k)

    retrieved_texts = []
    for i in indices[0]:
        retrieved_texts.append(all_chunks[i]['text'])

    return "\n\n".join(retrieved_texts)

In [14]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Load free, chat-ready open model
model_id = "HuggingFaceH4/zephyr-7b-beta"  # or try mistralai/Mistral-7B-Instruct
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype="auto")

qa_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

...

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00005-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cuda:0


In [15]:
def ask_chatbot(question):
    context = search_similar_chunks(question)

    prompt = f"""
Answer the question only using the context below.
If the answer is not present, say "I don't know".

Context:
{context}

Question:
{question}
Answer:
"""

    response = qa_pipeline(prompt, max_new_tokens=300, do_sample=True, temperature=0.3)[0]['generated_text']

    # Trim to answer only
    answer = response.split("Answer:")[-1].strip()

    return answer  # ✅ RETURN instead of print

In [17]:
question = input("Ask your sanitation-related question: ")
print(ask_chatbot(question))

Ask your sanitation-related question: What is sanitation?
Sanitation refers to the management of human feces at the household level, specifically the hygienic separation of human excreta from human contact. It includes the treatment of sewage, as well as the management of human waste and solid waste, and stormwater management. Sanitation workers are responsible for cleaning, maintaining, operating, or emptying sanitation technologies at any step of the sanitation chain. Improved sanitation is a term used to categorize types of sanitation for monitoring purposes, while the opposite, "unimproved sanitation," refers to facilities that do not hygienically separate human excreta from human contact.


In [18]:
pip install voila

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: jupyter-client
    Found existing installation: jupyter-client 6.1.12
    Uninstalling jupyter-client-6.1.12:
      Successfully uninstalled jupyter-client-6.1.12
  Attempting uninstall: jupyter-server
    Found existing installation: jupyter-server 1.16.0
    Uninstalling jupyter-server-1.16.0:
      Successfully uninstalled jupyter-server-1.16.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
notebook 6.5.7 requires jupyter-client<8,>=5.3.4, but yo

In [19]:
pip install notebook ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.5 MB/s eta 0:00:00
  Attempting uninstall: jupyter-client
    Found existing installation: jupyter_client 8.6.3
    Uninstalling jupyter_client-8.6.3:
      Successfully uninstalled jupyter_client-8.6.3


In [22]:
import ipywidgets as widgets
from IPython.display import display, clear_output

q_input = widgets.Text(placeholder="Ask a sanitation-related question...")
output = widgets.Output()

def on_submit(change):
    output.clear_output()
    with output:
        response = ask_chatbot(q_input.value)
        print(f"💬 Bot: {response}")

q_input.on_submit(on_submit)
display(q_input, output)

Text(value='', placeholder='Ask a sanitation-related question...')

Output()